+ Classificação de doenças (Rusty, Miner, Phoma) em folhas de café.
+ Rotina de Treinamento.
+ Fabio Cardoso.

In [ ]:
# Parâmetros

smp_sz           = 24      #Tamanho da amostra (de preferencia multiplo de 4) para cada epoch de treinamento.
qt_categs        = 3       #Quantidade distinta de labels/categorias.
qt_threads       = 4       #Paralelização
img_sq_size      = 384     #Tamanho para resizing das imagens.
epochs_traininig = 50      #Quantidade de epochs de treinamento.
file_ext         = '.jpg'  #Tipo de imagem.
use_base_model   = True    #Se faz uso de modelo pré-treinado.

In [ ]:
#Imports

#Geral
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from sklearn.metrics import log_loss

#images and tensorflow 
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import tensorflow as tf

#Processamento paralelo
import zlib
import pickle
import joblib
from multiprocessing import Pool

#Libs customizadas
import util0 #Preparação dos metadados
import util1f #Augmentation e preparação das imagens

# Keras
import keras
from keras import regularizers
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization

# Tensorflow
import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Attention
from tensorflow.keras import datasets, layers, models, initializers

np.random.seed(np.random.randint(999999999))
tf.random.set_seed(np.random.randint(999999999))

In [ ]:
# Paths

path_data  = '/kaggle/input/coffee-leaf-diseases/'
path_out   = '/kaggle/working/'
path_model = '/kaggle/input/coffee-leafs-classification-ok/'

In [ ]:
# Metadados - processamento paralelizado

imgnm_field = 'ImgName'
imdir_field = 'ImgDir'
lbl01_field =  None

qt_parts = 4 #Partes em paralelo (4 processadores)
    
df_meta_ini = pd.read_csv(path_data + 'train_classes.csv') #arquivo com labels
qt_per_part_train = int(df_meta_ini.shape[0] / qt_parts)
end_train = 0

#Popula parâmetros para chamadas em paralelo
parms = []
for i in range(qt_parts):
        ini_train = end_train
        end_train = ini_train + qt_per_part_train
        if i == qt_parts-1: end_train = None
        parms.append(dict({'path_data': path_data, 
                           'df_meta_ini_part': df_meta_ini[ini_train:end_train].copy()}))

pool = Pool(processes = qt_threads)
ret = pool.map(util0.meta_construct, parms)

df_meta = pd.concat(ret, axis=0)

In [ ]:
# Modeling

# hyper-parameters
k1  = 0.00005      #kregularizer1
b1  = 0.00001      #bregularizer1
k2  = 0.00005      #kregularizer2
b2  = 0.00001      #bregularizer2
do  = 0.12000      #drop-out rate
lr  = 0.00005      #learning rate

if use_base_model:
    #partindo de um modelo pré-treinado
    base_model = ConvNeXtBase(weights='imagenet', include_top=True, input_shape=(224, 224, 3))        
    for layer in base_model.layers: layer.trainable = True
            
def model4parms(k1,b1,k2,b2,do,lr):

    kernsz = (3,3,3)
    stride = (1,1,1)
    pollsz = (2,2,2)
    filters = 20

    input_layer = layers.Input(shape=(3, img_sq_size, img_sq_size, 3))

    if use_base_model:
        print('adicionando modelo de base')
        x = base_model(input_layer)

    x = layers.Conv3D(filters=filters, kernel_size=kernsz, strides=stride, padding="same",activation="relu", 
                      kernel_initializer=initializers.he_normal(), 
                      kernel_regularizer=regularizers.L2(k1),
                      bias_regularizer=regularizers.L2(b1))(input_layer)

    layers.BatchNormalization()(x)

    x = layers.MaxPooling3D(pool_size=pollsz, strides=pollsz, padding="same")(x)

    for _ in range(3):

        filters *= 2

        x = layers.Conv3D(filters=filters, kernel_size=kernsz, strides=stride, padding="same", activation="relu", 
                          kernel_initializer=initializers.he_normal(), 
                          kernel_regularizer=regularizers.L2(k1), 
                          bias_regularizer=regularizers.L2(b1))(x)

        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling3D(pool_size=pollsz, strides=pollsz, padding="same")(x)

    x = layers.Flatten()(x)

    x = layers.Dense(activation='relu', units=333, 
                     kernel_regularizer=regularizers.L2(k2), 
                     bias_regularizer=regularizers.L2(b2))(x)

    x = layers.Dropout(do)(x)

    x = layers.Dense(activation='relu', units=33, 
                     kernel_regularizer=regularizers.L2(k2), 
                     bias_regularizer=regularizers.L2(b2))(x)

    x = layers.Dropout(do)(x)

    outputs = [layers.Dense(2, activation='softmax', name=f'output{i}')(x) for i in range(qt_categs)]
    concatenated = layers.Concatenate(axis=1)(outputs)
    model = models.Model(inputs=input_layer, outputs=concatenated)

    optimizer1 = Adam(learning_rate=lr)

    model.compile(optimizer=optimizer1, loss=['categorical_crossentropy']*3, metrics='mean_absolute_error')

    return model

# Model instantiation
model_img = model4parms(k1,b1,k2,b2,do,lr)
print(model_img.summary())

In [ ]:
# Sets de treinamento, validação e holdout

# Separação dos sets por categoria (label) para fins de balanceamento 
df_meta_train_per_categ = []
df_meta_valid_per_categ = []
df_meta_tstng_per_categ = []
df_meta['categ'] = -1

for i in range(qt_categs+1):
    if i==0: qry="Miner==0 and Rust==0 and Phoma==0" #Sem doença
    if i==1: qry="Miner==1 and Rust==0 and Phoma==0" #Miner
    if i==2: qry="Miner==0 and Rust==1 and Phoma==0" #Rust
    if i==3: qry="Miner==0 and Rust==0 and Phoma==1" #Phoma
    df_categ = df_meta.query(qry).copy()
    df_categ['categ'] = i

    #Separa 60% para treinamento. Do restante, 60% para validação. Restante para holdout.
    df_train = df_categ.sample(frac=.6, replace=False)
    id_remng = [x for x in df_categ['ImgName'].values if x not in df_train['ImgName'].values]
    df_valid = df_categ[df_categ['ImgName'].isin(id_remng)].sample(frac=.6, replace=False)
    id_remng = [x for x in df_categ['ImgName'].values if x not in df_train['ImgName'].values and x not in df_valid['ImgName'].values]
    df_tstng = df_categ[df_categ['ImgName'].isin(id_remng)]
    df_meta_train_per_categ.append(df_train)
    df_meta_valid_per_categ.append(df_valid)
    df_meta_tstng_per_categ.append(df_tstng)

# Salva datasets para eventual continuacao de treinamento
df_meta_train = pd.concat(df_meta_train_per_categ, axis=0)
df_meta_train.to_csv('df_meta_train.csv')
df_meta_valid = pd.concat(df_meta_valid_per_categ, axis=0)
df_meta_valid.to_csv('df_meta_valid.csv')
df_meta_tstng = pd.concat(df_meta_tstng_per_categ, axis=0)
df_meta_tstng.to_csv('df_meta_tstng.csv')

In [ ]:
# Epochs de treinmanto

best_mea =  +9999

for h in range(epochs_traininig):

    # Sampling
    print('Sampling #', h)

    df_meta_train_smp = []
    for i in range(qt_categs+1):
        df_meta_train_smp.append(df_meta_train_per_categ[i].sample(n=smp_sz, replace=True))

    df_meta_valid_smp = []
    for i in range(qt_categs+1):
        df_meta_valid_smp.append(df_meta_valid_per_categ[i].sample(n=smp_sz, replace=True))

    df_meta_train_smp = pd.concat(df_meta_train_smp, axis=0).sample(frac=1, replace=False)
    df_meta_valid_smp = pd.concat(df_meta_valid_smp, axis=0).sample(frac=1, replace=False)

    # Preparação das imagens
    print('Preparação das imagens')

    parms = []
    end_train = 0
    end_valid = 0

    qt_per_part_train = int(df_meta_train_smp.shape[0] / qt_parts)
    qt_per_part_valid = int(df_meta_valid_smp.shape[0] / qt_parts)

    for i in range(qt_parts):

            ini_train = end_train
            end_train = ini_train + qt_per_part_train

            ini_valid = end_valid
            end_valid = ini_valid + qt_per_part_valid

            if i == qt_parts-1:
                end_train = end_valid = None

            parms.append(dict({'df_meta_smp':df_meta_train_smp[ini_train:end_train], 
                               'path_data':path_data, 'setx':'train', 'file_ext':file_ext, 
                               'rodada':i, 'img_sq_size':img_sq_size, 'imgnm_field':imgnm_field,
                               'imdir_field':imdir_field,'lbl01_field':lbl01_field, 'aug':True, 'seg':False}))

            parms.append(dict({'df_meta_smp':df_meta_valid_smp[ini_valid:end_valid], 
                               'path_data':path_data, 'setx':'valid', 'file_ext':file_ext, 
                               'rodada':i, 'img_sq_size':img_sq_size, 'imgnm_field':imgnm_field,
                               'imdir_field':imdir_field,'lbl01_field':lbl01_field, 'aug':True, 'seg':False}))

    pool = Pool(processes = qt_threads)
    ret = pool.map(util1f.smp2vol, parms)

    # Descompacta resultados
    print('Descompacta resultados')

    for i in range(qt_parts *1):
        ret[i][0] = pickle.loads(zlib.decompress(ret[i][0]))

    Xt = np.vstack([ret[i][0] for i in range(len(ret)) if ret[i][2]=='train'])
    Xv = np.vstack([ret[i][0] for i in range(len(ret)) if ret[i][2]=='valid'])

    # Prepara labels para softmax
    print('Prepara labels para softmax')

    Yt = []
    Yv = []
    for i in range(len(ret)):
        if ret[i][2]=='train': Yt = Yt + ret[i][1] 
        if ret[i][2]=='valid': Yv = Yv + ret[i][1] 
    Yt = np.array(Yt)
    Yv = np.array(Yv)

    #Treina
    print('Treina')

    stt = model_img.fit(Xt, Yt, epochs=1, batch_size=1,
                validation_data=(Xv, Yv), verbose=2, shuffle=False)

    mea = stt.history.get("val_mean_absolute_error")[0]

    if mea < best_mea:
        best_mea = mea
        qtd_saved +=1
        model_img.save('best_model.h5')
        print('best measure', mea)

    #limpa memoria
    gc.collect()